In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:2]:
        print(os.path.join(dirname, filename))

/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv
/kaggle/input/prostate-cancer-grade-assessment/test.csv
/kaggle/input/prostate-cancer-grade-assessment/train_images/2673584f9398ce0acb21a86a1a711088.tiff
/kaggle/input/prostate-cancer-grade-assessment/train_images/5d46da93924a4b15472581eb39658309.tiff
/kaggle/input/prostate-cancer-grade-assessment/train_label_masks/9bce8bb47c22ab502ed7266e2e3762c0_mask.tiff
/kaggle/input/prostate-cancer-grade-assessment/train_label_masks/124a0616099409f5b9aaedfd3ac3ab6d_mask.tiff
/kaggle/input/nhan-hdf5/img_dtb1.h5
/kaggle/input/nhan-hdf5/img_dtb2.h5
/kaggle/input/h5-files/full_data_coordinate.h5


In [2]:
import os

# There are two ways to load the data from the PANDA dataset:
# Option 1: Load images using openslide
import openslide
# Option 2: Load images using skimage (requires that tifffile is installed)
import skimage.io
import random
import seaborn as sns
import cv2

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import PIL
from IPython.display import Image, display

# Plotly for the interactive viewer (see last section)
import plotly.graph_objs as go

# Location of the training images

BASE_PATH = '../input/prostate-cancer-grade-assessment'

# image and mask directories
data_dir = f'{BASE_PATH}/train_images'
mask_dir = f'{BASE_PATH}/train_label_masks'


# Location of training labels
train = pd.read_csv(f'{BASE_PATH}/train.csv').set_index('image_id')
test = pd.read_csv(f'{BASE_PATH}/test.csv')

train_labels = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv').set_index('image_id')

submission = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')

data_dirname = [name.replace('.tiff', '') for name in os.listdir(data_dir)]
id_dir = [name.replace('_mask.tiff', '') for name in os.listdir(mask_dir)]
id_dir.sort()
id_dir.remove('3790f55cad63053e956fb73027179707')
img_id_mutual_name = id_dir
img_id_mutual_name[:5], len(img_id_mutual_name)

(['0005f7aaab2800f6170c399693a96917',
  '000920ad0b612851f8e01bcc880d9b3d',
  '0018ae58b01bdadc8e347995b69f99aa',
  '001c62abd11fa4b57bf7a6c603a11bb9',
  '001d865e65ef5d2579c190a0e0350d8f'],
 10515)

In [3]:
import tables
import h5py
import deepdish as dd
import tables
import h5py
from sys import getsizeof
import deepdish as dd

In [4]:
import deepdish as dd

df = dd.io.load('/kaggle/input/h5-files/full_data_coordinate.h5')
len(df)//36, len(df[0]), df[0]

(9790, 5, ['0005f7aaab2800f6170c399693a96917', 13312, 13824, 7168, 7680])

In [5]:
def load_data_and_mask(ID, coordinates, level = 1):
    """
    
    """
    data_img = skimage.io.MultiImage(os.path.join(data_dir, f'{ID}.tiff'))[level]
    mask_img = skimage.io.MultiImage(os.path.join(mask_dir, f'{ID}_mask.tiff'))[level]
    coordinates = [coordinate // 2**(2*level) for coordinate in coordinates]
    data_tile = data_img[coordinates[0]: coordinates[1], coordinates[2]: coordinates[3], :]
    mask_tile = mask_img[coordinates[0]: coordinates[1], coordinates[2]: coordinates[3], :]
    data_tile = cv2.resize(data_tile, (512, 512))
    mask_tile = cv2.resize(mask_tile, (512, 512))
    del data_img, mask_img
    
    # Load and return small image
    return data_tile, mask_tile

%time a, b = load_data_and_mask(df[0][0], df[0][1:], 1)
a.shape, b.shape

CPU times: user 354 ms, sys: 150 ms, total: 504 ms
Wall time: 564 ms


((512, 512, 3), (512, 512, 3))

In [6]:
from torch.utils.data import Dataset, DataLoader
import torch

class PANDADataset(Dataset):
    def __init__(self, df, level=1, transform=None):
        self.df = df
        self.level = level
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index, level = 1):
        ID = self.df[index][0]
        coordinate = self.df[index][1: ]
        image, mask = load_data_and_mask(ID, coordinate, level)
        
        return torch.tensor(image).permute(2, 0, 1), torch.tensor(mask).permute(2, 0, 1)[0]
    
cls = PANDADataset(df[:800], 1)
%time cls[0][0].size(), cls[0][1].size(), len(cls)

CPU times: user 676 ms, sys: 260 ms, total: 936 ms
Wall time: 952 ms


(torch.Size([3, 512, 512]), torch.Size([512, 512]), 800)

In [7]:
dataLoader = DataLoader(cls, batch_size=4, shuffle=True, num_workers=4)

for i_batch, data in enumerate(dataLoader):
    inputs, labels = data
    print(i_batch, ': \t', inputs.size(), ', ', labels.size())
    del inputs, labels
    if i_batch == 4:
        break

0 : 	 torch.Size([4, 3, 512, 512]) ,  torch.Size([4, 512, 512])
1 : 	 torch.Size([4, 3, 512, 512]) ,  torch.Size([4, 512, 512])
2 : 	 torch.Size([4, 3, 512, 512]) ,  torch.Size([4, 512, 512])
3 : 	 torch.Size([4, 3, 512, 512]) ,  torch.Size([4, 512, 512])
4 : 	 torch.Size([4, 3, 512, 512]) ,  torch.Size([4, 512, 512])


In [8]:
# Adapted from https://discuss.pytorch.org/t/unet-implementation/426

import torch
from torch import nn
import torch.nn.functional as F


class UNet(nn.Module):
    def __init__(self, in_channels=1, n_classes=2, depth=5, wf=6, padding=False,
                 batch_norm=False, up_mode='upconv'):
        """
        Implementation of
        U-Net: Convolutional Networks for Biomedical Image Segmentation
        (Ronneberger et al., 2015)
        https://arxiv.org/abs/1505.04597
        Using the default arguments will yield the exact version used
        in the original paper
        Args:
            in_channels (int): number of input channels
            n_classes (int): number of output channels
            depth (int): depth of the network
            wf (int): number of filters in the first layer is 2**wf
            padding (bool): if True, apply padding such that the input shape
                            is the same as the output.
                            This may introduce artifacts
            batch_norm (bool): Use BatchNorm after layers with an
                               activation function
            up_mode (str): one of 'upconv' or 'upsample'.
                           'upconv' will use transposed convolutions for
                           learned upsampling.
                           'upsample' will use bilinear upsampling.
        """
        super(UNet, self).__init__()
        assert up_mode in ('upconv', 'upsample')
        self.padding = padding
        self.depth = depth
        prev_channels = in_channels
        self.down_path = nn.ModuleList()
        for i in range(depth):
            self.down_path.append(UNetConvBlock(prev_channels, 2**(wf+i),
                                                padding, batch_norm))
            prev_channels = 2**(wf+i)

        self.up_path = nn.ModuleList()
        for i in reversed(range(depth - 1)):
            self.up_path.append(UNetUpBlock(prev_channels, 2**(wf+i), up_mode,
                                            padding, batch_norm))
            prev_channels = 2**(wf+i)

        self.last = nn.Conv2d(prev_channels, n_classes, kernel_size=1)

    def forward(self, x):
        blocks = []
        for i, down in enumerate(self.down_path):
            x = down(x)
            if i != len(self.down_path)-1:
                blocks.append(x)
                x = F.avg_pool2d(x, 2)

        for i, up in enumerate(self.up_path):
            x = up(x, blocks[-i-1])

        return self.last(x)


class UNetConvBlock(nn.Module):
    def __init__(self, in_size, out_size, padding, batch_norm):
        super(UNetConvBlock, self).__init__()
        block = []

        block.append(nn.Conv2d(in_size, out_size, kernel_size=3,
                               padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        block.append(nn.Conv2d(out_size, out_size, kernel_size=3,
                               padding=int(padding)))
        block.append(nn.ReLU())
        if batch_norm:
            block.append(nn.BatchNorm2d(out_size))

        self.block = nn.Sequential(*block)

    def forward(self, x):
        out = self.block(x)
        return out


class UNetUpBlock(nn.Module):
    def __init__(self, in_size, out_size, up_mode, padding, batch_norm):
        super(UNetUpBlock, self).__init__()
        if up_mode == 'upconv':
            self.up = nn.ConvTranspose2d(in_size, out_size, kernel_size=2,
                                         stride=2)
        elif up_mode == 'upsample':
            self.up = nn.Sequential(nn.Upsample(mode='bilinear', scale_factor=2),
                                    nn.Conv2d(in_size, out_size, kernel_size=1))

        self.conv_block = UNetConvBlock(in_size, out_size, padding, batch_norm)

    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[:, :, diff_y:(diff_y + target_size[0]), diff_x:(diff_x + target_size[1])]

    def forward(self, x, bridge):
        up = self.up(x)
        crop1 = self.center_crop(bridge, up.shape[2:])
        out = torch.cat([up, crop1], 1)
        out = self.conv_block(out)

        return out

In [9]:
# --- unet params
#these parameters get fed directly into the UNET class, and more description of them can be discovered there

n_classes= 6    #number of classes in the data mask that we'll aim to predict


in_channels= 3  #input channel of the data, RGB = 3
padding= True   #should levels be padded
depth= 5       #depth of the network 
wf= 2           #wf (int): number of filters in the first layer is 2**wf, was 6
up_mode= 'upconv' #should we simply upsample the mask, or should we try and learn an interpolation 
batch_norm = True #should we use batch normalization between the layers

# --- training params
batch_size=4
patch_size=512
num_epochs = 10 # 100

edge_weight = 1.1 #edges tend to be the most poorly segmented given how little area they occupy in the training set, this paramter boosts their values along the lines of the original UNET paper
phases = ["train","val"] #how many phases did we create databases for?
validation_phases= ["val"] #when should we do valiation? note that validation is time consuming, so as opposed to doing for both training and validation, we do it only for vlaidation at the end of the epoch

In [10]:
gpuid=0
#specify if we should use a GPU (cuda) or only the CPU
if(torch.cuda.is_available()):
    print(torch.cuda.get_device_properties(gpuid))
    torch.cuda.set_device(gpuid)
    device = torch.device(f'cuda:{gpuid}')
else:
    device = torch.device(f'cpu')


_CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [11]:
#build the model according to the paramters specified above and copy it to the GPU. finally print out the number of trainable parameters
model = UNet(n_classes=n_classes, in_channels=in_channels, padding=padding,depth=depth,wf=wf, up_mode=up_mode, batch_norm=batch_norm).to(device)
print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")

total params: 	122486


In [12]:
optim = torch.optim.Adam(model.parameters()) #adam is going to be the most robust
criterion = nn.CrossEntropyLoss(reduce=False)

/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [13]:
for epoch in range(num_epochs):
    #model.train()  # Set model to training mode
    running_loss = 0.0
    
    for i, data in enumerate(dataLoader, 0):
        inputs, labels = data
        inputs = inputs.to(device,dtype=torch.float) 
        labels = labels.type('torch.LongTensor').to(device)
        ##with torch.set_grad_enabled(True):
        # zero the parameter gradients
        optim.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.sum().backward()
        optim.step()
        # print statistics
        
        running_loss += loss.mean()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 1.569
[2,   200] loss: 1.324
[3,   200] loss: 1.157
[4,   200] loss: 1.095
[5,   200] loss: 1.058
[6,   200] loss: 1.048
[7,   200] loss: 1.031
[8,   200] loss: 1.025
[9,   200] loss: 1.018
[10,   200] loss: 1.012
Finished Training
